# Análisis del riesgo de incumplimiento de los prestatarios


# Contenido <a id='back'></a>

* [Introducción](#intro)
* [1. Exploración de datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [2. Transformación de datos](#data_preprocessing)
    * [2.1 Trabajar con valores ausentes](#missing_values)
* [3 Clasificación de datos](#data_clasification)
* [4. Comprobación de las hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: ¿Existe una correlación entre tener hijos y pagar a tiempo?](#hipo1)
    * [3.2 Hipótesis 2: ¿Existe una correlación entre la situación familiar y el pago a tiempo?](#hipo2)
    * [3.3 Hipótesis 3: ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?](#hipo3)
    * [3.3 Hipótesis 4: ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?](#hipo4)
* [Conclusiones generales](#end)


# Introducción <a id='intro'></a>

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

# Objetivos

Prueba de hipótesis:

1. ¿Existe una correlación entre tener hijos y pagar a tiempo?
2. ¿Existe una correlación entre la situación familiar y el pago a tiempo?
3. ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?
4. ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?


# 1. Exploración de datos <a id='data_review'></a>

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Abrimos los datos desde nuestro archivo y los examinamos

In [1]:
# cargar todas las librerías
import pandas as pd

# Carga los datos
df=pd.read_csv('/datasets/credit_scoring_eng.csv')

In [2]:
#Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
print(df.shape)

(21525, 12)


In [3]:
# vamos a mostrar las primeras filas N

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding



Podemos ver dos problemas con los datos en las columnas:
1. En la columna days_employed encontramos valores negativos
2. En la columna education observamos duplicados implícitos

Los valores negativos pueden deberse a un error de factor humano

In [4]:
# Obtener información sobre los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Encontramos valores ausentes en dos columnas: days_employed y total_income. De hecho, curiosamente la misma cantidad de valores ausentes, por lo que se revisará si estos valores coinciden para ambas columnas

In [5]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

df[df.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Los valores NaN que aparecen ausentes en las ambas columnas coinciden. Hemos comprobado que son la misma cantidad de números exacto.

In [6]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Conclusión intermedia**

Con esto comprobamos que los valores ausentes coinciden en número y posición entre estas dos columnas:days_employed y total_income.

Calcularemos el porcentaje que representan estos valores ausentes 

%=(2174x100)/21525

 Representa al 10,09%
 
A pesar que no es una porción de datos considerablemente grande, podría afectar nuestros resultados por lo que se procederá a completar los valores ausentes. 

En este análisis se ha identificado que los datos ausentes podrían deberse a una característica específica del cliente, la mayoría de valores ausentes se encuentran en personas con educación secundaria. 

1. Se debe investigar si realmente se relacionan los valores ausentes con personas de educación secundaria.
2. Se debe deducir cuál es la razón por la que no tenemos estos valores

In [7]:
#Primero corregiremos a minúsculas los datos de la columna 'education' para ayudarnos a identificar mejor con ayuda de nuestra principal característica

df['education']=df['education'].str.lower()
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [8]:
#Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausente

secondary_education= df[df['education']=='secondary education']
secondary_education[secondary_education.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21426,0,NaN,49,secondary education,1,married,0,F,employee,1,NaN,property
21489,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [9]:
# Comprobación de la distribución

bachelor_degree=df[df['education']!='secondary education']
bachelor_degree[bachelor_degree.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
90,2,NaN,35,bachelor's degree,0,married,0,F,employee,0,NaN,housing transactions
94,1,NaN,34,bachelor's degree,0,civil partnership,1,F,business,0,NaN,having a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate
21294,0,NaN,45,bachelor's degree,0,married,0,F,employee,0,NaN,university education
21432,1,NaN,38,some college,2,unmarried,4,F,employee,0,NaN,housing transactions
21463,1,NaN,35,bachelor's degree,0,civil partnership,1,M,employee,0,NaN,having a wedding


Tenemos que 1540 personas con educación secundaria que poseen datos ausentes en education y total_income.
Tenemos que 634 personas con some college y bachelor's degree complementan el total de personas con datos ausentes en las mismas columnas

De el total que suman de datos ausentes, que es 2174. Con esta información podemos deducir que hay un patrón en la falta de información lo que podría advertir falta de información por voluntad y no por error.

Hemos denotado que como se pensó anteriormente, no todos los datos pertenecen a las personas con 'secondary education' aunque si es una gran porción del total de datos ausentes.
A continuación descubriremos más

In [10]:
# Comprobando la distribución en el conjunto de datos de educación secundaria

df[df['education']=='secondary education']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
7,0,-152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [11]:
# Comprobamos otras razones y patrones que podrían llevar a valores ausentes

df[df.total_income.isnull()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Calculemos el porcentaje de personas a las que representa del total el número de personas con valores ausentes en secondary education 
(1540x100)/21525= 7,15% 

# Conclusiones <a id='data_review_conclusions'></a>

Debido a que de todos los valores ingresados de 'secondary education', los que coinciden con valores ausentes en las columnas 'total_income' y 'days_employed' pertenecen al 7% del total de personas con valores faltantes. Además los 634 restantes también pertencen a un grupo muy pequeño. Podemos deducir que estos valores se encuentran ausentes por alguno(s) de los siguientes motivos:

-Los clientes no revelaron esta información por voluntad propia

-Los clientes no otorgaron esta información porque no estaban seguros de sus ingresos, ni sus días trabajados

-La información faltante no fue escrita en la base de datos por parte del Banco

Para continuar ahora necesitamos reprocesar los datos
1. Revisar duplicados implícitos (arriba corregimos los valores en mayúscula de la columna 'education')
2. Nos ocuparemos de los valores ausentes. Debido a que son valores cuantitativos necesitamos analizar que valores reemplazarán aquellos datos ausentes.
3. Debemos revisar y modificar los datos que se encuentran en un formato erróneo (negativos a positivos)


# 2. Transformación de datos <a id='data_preprocessing'></a>

Vamos a repasar las columnas, una por una, verificando que los datos en ellas sean correctos

In [12]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir
#Esta información la corregimos en la primera parte, verificamos que haya cambiado correctamente

df['education'].sort_values().unique()

array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

In [13]:
# Veamos la distribución de los valores en la columna `children`
df['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

Aquí podemos observar dos valores incongruentes: -1 y 20. No podemos tener una cantidad negativa en niños y 20 es un número particularmente alto.

Revisamos la tabla para contabilizar el número de personas que poseen estos valores

In [14]:
# Vemos los valores que incluyen -1 
df[df['children']==-1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
705,-1,-902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
742,-1,-3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
800,-1,349987.852217,54,secondary education,1,unmarried,4,F,retiree,0,13806.996,supplementary education
941,-1,NaN,57,secondary education,1,married,0,F,retiree,0,NaN,buying my own car
1363,-1,-1195.264956,55,secondary education,1,married,0,F,business,0,11128.112,profile education
1929,-1,-1461.303336,38,secondary education,1,unmarried,4,M,employee,0,17459.451,purchase of the house
2073,-1,-2539.761232,42,secondary education,1,divorced,3,F,business,0,26022.177,purchase of the house
3814,-1,-3045.290443,26,secondary education,1,civil partnership,1,F,civil servant,0,21102.846,having a wedding
4201,-1,-901.101738,41,secondary education,1,married,0,F,civil servant,0,36220.123,transactions with my real estate


In [15]:
# Vemos los valores que incluyen 20 
df[df['children']==20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


Vemos que el valor -1 puede deberse a un error de factor humano, adicional el valor 20 es un valor muy alto y aunque sería posible tener esa cantidad de hijos, no creo que se cumpla para tantas personas por lo que se decide a dejarlo en 2 porque puede deberse a un error de tipeo

In [16]:
# Cambiar los datos y comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

df['children']= df['children'].replace(-1,1)
df['children']= df['children'].replace(20,2)  
    
df['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5])

Ahora comprobaremos los datos de la columna 'days_employed'

In [17]:
# Encuentra datos problemáticos en `days_employed`, si existen
df['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

Vemos una gran cantidad de números negativos. Y también recordamos que en esta columna tenemos datos ausentes.Quizá hubo algún error al leer la cantidad, por ahora lo cambiaremos a positivo los negativos

In [18]:
# Aborda los valores problemáticos, si existen.
# Comprueba el resultado - asegúrate de que esté arreglado
df['days_employed']=df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [19]:
# Debemos corregir el número de días, comprobamos la edad max.
df['dob_years'].max()

75

Vemos que la edad máxima es 75 años. Muchos se retiran a los 65 años por lo que podemos asumir un máximo de 50 años trabajados o 18250 días

In [20]:
# Revisemos los valores mayores a 18250.
df[df['days_employed'] >18250]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


In [21]:
# Creamos una tabla donde se multiplique la cantidad de años con 200 días laborables(aprox) por año a partir de los 20 para que nos de un aproximado de años según la edad de cada persona
# Ahora reemplazamos los valores que cumplan la condición de ser mayores a 18250 días
df['days_employed_right']=(df['dob_years']-20)*200
df.loc[df['days_employed']>18250, 'days_employed']=df.loc[df['days_employed']>18250, 'days_employed_right']
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,4400
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,3200
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,2600
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,2400
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,6600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,4600
21521,0,9400.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,9400
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,3600
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,3600


In [22]:
# No eliminamos la columna adicional porque nos servirá para completar los valores ausentes en 'days_employed'
# Comprobamos que no tengamos valores mayores a 18250 en 'days_employed'
df[df['days_employed'] >18250]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right


Ahora echemos un vistazo a la edad de clientes para ver si hay algún problema allí. 

In [23]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Encontramos que uno de los valores dice 0, lo que sería imposible.

In [24]:
# Revisa `dob_years` en busca de valores sospechosos
df[df['dob_years']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right
99,0,-4000.000000,0,secondary education,1,married,0,F,retiree,0,11406.644,car,-4000
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions,-4000
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation,-4000
578,0,-4000.000000,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property,-4000
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car,-4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing,-4000
20462,0,-4000.000000,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house,-4000
20577,0,-4000.000000,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property,-4000
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate,-4000


Vemos que tenemos a 101 personas con 0 años lo que representa al (101x100)/21525=0,46% es una cantidad mínima, por lo que podríamos eliminar estos datos, sin embargo en el análisis de este informe NO influye la edad de las personas, pero SÍ sus ingresos, deudas, cantidad de hijos y otros factores. Ahora, rellenar estos valores ausentes me ayudará a evitar la pérdida de información relevante para el estudio

In [25]:
# Hallaremos un valor categórico para el reemplazo

age_avg=df.loc[df['dob_years']!=0, 'dob_years'].median()
print(age_avg)

43.0


In [26]:
# Hallamos un valor categórico para los retirados
retiree = df[(df['dob_years']!=0) & (df['income_type']=='retiree')]
print(retiree['dob_years'].median())

60.0


In [27]:
# Reemplazamos los valores primero en los retiree que cumplan con la condición
df.loc[(df['income_type'] == 'retiree') & ( df['dob_years'] == 0) , 'dob_years'] = 60

#Luego con las demás personas con edad 0
df.loc[df['dob_years']==0, 'dob_years']=43

In [28]:
# Comprobamos el resultado, no nos debe quedar nadie con edad 0
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 60, 59, 29, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Ahora revisamos la columna `family_status`.

In [29]:
# Veamos los valores de la columna

df['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [30]:
# Aborda los valores problemáticos en `family_status`

df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

Ahora revisamos la columna `gender`. 

In [31]:
# Veamos los valores en la columna
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [32]:
# Cambiamos el valor XNA por X (No se encuentra dentro de ninguna identidad de género)
df['gender']= df['gender'].replace('XNA', 'X')

In [33]:
# Comprueba el resultado
df['gender'].unique()

array(['F', 'M', 'X'], dtype=object)

Ahora vamos a revisar la columna `income_type`

In [34]:
# Veamos los valores en la columna
df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

Ahora veamos si hay duplicados en nuestros datos.

Ahora revisaremos la columna purpose

In [35]:
# Última comprobación para ver si tenemos duplicados
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Podemos denotar algunos duplicados implícitos, por lo que los vamos a cambiar para un mejor análisis.

In [36]:
# Vamos a crear una función para reemplazar los duplicados implícitos

def replace_wrong_values(wrong_values,correct_value):
    for values in wrong_values:
        df['purpose']= df['purpose'].replace(values,correct_value)
        
# Eliminamos los duplicados implícitos
wedding=['to have a wedding','having a wedding', 'wedding ceremony']
car=['cars', 'buying a second-hand car', 'car purchase', 'buying my own car', 'second-hand car purchase', 'car','to own a car', 'purchase of a car', 'to buy a car']
house=['house purchase','housing renovation','purchase of my own house', 'housing','housing transactions','purchase of the house', 'purchase of the house for my family','buy real estate','construction of own property','property','building a property', 'buying property for renting out']
education=['supplementary education','to become educated','getting an education','to get a supplementary education','getting higher education', 'profile education', 'university education', 'going to university']
real_state=['building a real estate','real estate transactions','buy residential real estate', 'buy commercial real estate','transactions with commercial real estate','transactions with my real estate']

wedding_1='wedding'
car_1='car'
house_1='house'
education_1='education'
real_state_1='real estate'

replace_wrong_values(wedding, wedding_1)
replace_wrong_values(car, car_1)
replace_wrong_values(house, house_1)
replace_wrong_values(education, education_1)
replace_wrong_values(real_state, real_state_1)


df['purpose'].unique()

array(['house', 'car', 'education', 'wedding', 'real estate'],
      dtype=object)

In [37]:
# Comprobar los duplicados

df.duplicated().sum()

328

In [38]:
# Aborda los duplicados
df=df.drop_duplicates().reset_index(drop=True)

In [39]:
# Última comprobación para ver si tenemos duplicados
df.duplicated().sum()

0

In [40]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21197 entries, 0 to 21196
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21197 non-null  int64  
 1   days_employed        19351 non-null  float64
 2   dob_years            21197 non-null  int64  
 3   education            21197 non-null  object 
 4   education_id         21197 non-null  int64  
 5   family_status        21197 non-null  object 
 6   family_status_id     21197 non-null  int64  
 7   gender               21197 non-null  object 
 8   income_type          21197 non-null  object 
 9   debt                 21197 non-null  int64  
 10  total_income         19351 non-null  float64
 11  purpose              21197 non-null  object 
 12  days_employed_right  21197 non-null  int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


Detectamos tres problemas con los datos:

- Errores de factor humano para algunas cantidades en negativo en days_employed, también valores absurdamente grandes que se cambiaron por valores representativos
- Valores ausentes
- Duplicados obvios e implícitos

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender. La correción de algunos de los datos en nuestra tabla de 21525 filas ha hecho que se reduzca a 21453 entradas que describen un conjunto de datos sin duplicados obvios y sin valores negativos. También utilizamos valores representativos para los datos 0 en la columna de dob_years. Y en la columna days_employed para los 3446 datos que representaban muchos años. Esto nos ayudará a no perder datos valiosos para nuestro análisis final.

Pero todavía tenemos que ver si los valores ausentes en nuestras columnas: days_employed y total_income afectan o no a nuestros cálculos.


# Trabajar con valores ausentes <a id='missing_values'></a>

**Restaurar valores ausentes en `total_income`**

Tenemos valores ausentes en la columna 'total_income'. Primero crearemos una columna que clasifique por edad a las personas. Esto nos puede ayudar a calcular valores para el ingreso total. Lo mejor que podemos hacer es utilizar valores representativos para los valores ausentes en total income.

In [41]:
# Vamos a escribir una función que calcule la categoría de edad

def age_group(age):
    
    if age <32:
        return'young'
    if age <46:
        return'young adult'
    if age <= 60:
        return 'adult'
    return 'elderly'

In [42]:
# Vemos si la función funciona bien
print(age_group(35))

young adult


In [43]:
# Crear una nueva columna basada en la función

df['age_group']=df['dob_years'].apply(age_group)

In [44]:
# Crear una nueva columna basada en la función

df.age_group.value_counts()

young adult    7935
adult          6924
young          4246
elderly        2092
Name: age_group, dtype: int64

In [45]:
# Comprobamos los valores en la nueva columna
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,4400,young adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,3200,young adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,2600,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,2400,young adult
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,6600,adult


Los ingresos dependen de varios factores. Uno de los principales es el trabajo. Mas adelante veremos que valor representativo será la mejor opción para rellenar estos valores ausentes

Vamos a crear una tabla que no posea valores ausentes. Esto será para encontrar los valores representativos

In [46]:
# Creamos una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
df_new=df[df['days_employed'].notnull()]
df_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,4400,young adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,3200,young adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,2600,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,2400,young adult
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,6600,adult


In [47]:
# Examinamos los valores medios de los ingresos en función a su grupo de edad
log=df_new[['total_income','age_group']]
log.groupby('age_group').mean()


,total_income
age_group,
adult,26367.231970
elderly,23057.777452
young,26074.635654
young adult,28522.419496


In [48]:
# Examina los valores medianos de los ingresos en función a su grupo de edad
logs=df_new[['total_income','age_group']]
logs.groupby('age_group').median()

,total_income
age_group,
adult,22852.0070
elderly,19637.0560
young,23041.9840
young adult,24818.4925


In [49]:
# Examina los valores medios de los ingresos en función a su income type
log_1=df_new[['total_income','income_type']]
log_1.groupby('income_type').mean()

,total_income
income_type,
business,32386.793835
civil servant,27343.729582
employee,25820.841683
entrepreneur,79866.103000
paternity / maternity leave,8612.661000
retiree,21940.394503
student,15712.260000
unemployed,21014.360500


In [50]:
# Examina los valores medianos de los ingresos en función a su income type
logs_1=df_new[['total_income','income_type']]
logs_1.groupby('income_type').median()

,total_income
income_type,
business,27577.2720
civil servant,24071.6695
employee,22815.1035
entrepreneur,79866.1030
paternity / maternity leave,8612.6610
retiree,18962.3180
student,15712.2600
unemployed,21014.3605


Hemos hecho dos razonamientos para total_income:
1. Tomamos en cuenta el grupo de edad, como podemos apreciar el grupo de young adult y young poseen ingresos mayores que los eldery por ejemplo. Sin embargo entre la mediana y la media, observamos que la media nos da un valor tanto mayor que la mediana

2. Para el segundo análisis tomamos en cuenta el tipo de ingreso, lo cual me parece más acorde para el ingreso económico. Para completar los datos ausentes de total_income utilizaremos la mediana en función de income_type, debido a que se toma mejor consideración para los valores atípicos que encontramos en esta columna.

Los valores representativos en la tabla df y df_new no cambian por lo que mas adelante usaremos df para los valores representativos


In [51]:
#  Escribimos una función que usaremos para completar los valores ausentes
# 1. Agrupamos por la columna que deseamos evaluar y encontramos la mediana, también colocamos índice
# 2. Utilizamos merge para unir esta columna a nuestro dataframe original
# 3. Ahora reemplazamos los valores NaN

def fill_NaN_values_median (data, merge_columns, column_input):
    
    name_column=f'median_{column_input}'
   
    logs=data[[column_input, merge_columns]]
    median_by_group=logs.groupby(merge_columns).median().reset_index()
    median_by_group.rename(columns={column_input:name_column}, inplace=True)

    df_merge=data.merge(median_by_group, on=merge_columns, how='left')
    df_merge.loc[df_merge[column_input].isna(), column_input]=df_merge.loc[df_merge[column_input].isna(), name_column]
    
    df_merge=df_merge.drop([name_column], axis=1)
    
    return df_merge

In [52]:
# Comprobamos si funciona
df_merge=fill_NaN_values_median(df,'income_type', 'total_income')
df_merge.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_right,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,4400,young adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,3200,young adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,2600,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,2400,young adult
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,6600,adult


In [53]:
# Comprobamos los valores ausente en total_income
df_merge.isna().sum()

children                  0
days_employed          1846
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
days_employed_right       0
age_group                 0
dtype: int64

No hemos encontrado errores al asignar categoría por edad ni al momento de rellenar los valores ausentes por la mediana, nuestra nueva tabla lleva el nombre de df_merge

In [54]:
# Comprobar el número de entradas en las columnas
print(df_merge.shape)
df_merge.total_income

(21197, 14)


0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21192    35966.698
21193    24959.969
21194    14347.610
21195    39054.888
21196    13127.587
Name: total_income, Length: 21197, dtype: float64

**Restaurar valores en `days_employed`**

Una de las primeras ideas que se viene a la cabeza cuando deseamos rellenar días de trabajo, es el tipo de trabajo que se tiene y la edad. Anteriormente utilizamos la edad para crear una columna con un valor aproximado de días de trabajo dependiendo de la edad. Ahora la usaremos para completar los valores ausentes de esta columna.

In [55]:
# Examinamos los valores de la mediana en función a su tipo de ingreso
dt=df_merge[['days_employed','age_group']]
dt.groupby('age_group').median()

,days_employed
age_group,
adult,3714.775894
elderly,8600.000000
young,1074.326599
young adult,1768.767822


In [56]:
# Examinamos los valores medios de los ingresos en función a su tipo de ingreso
dt_1=df_merge[['days_employed','age_group']]
dt_1.groupby('age_group').mean()

,days_employed
age_group,
adult,4293.211145
elderly,7898.878447
young,1327.556916
young adult,2337.160380


Como valor representativo para los grupos escogidos, vamos a utilizar los valores de la columna creada anteriormente, 'days_employed_right'. Este valor esta relacionado a la edad del cliente.

In [57]:
# Ahora reemplazamos los valores NaN
df_merge.loc[df['days_employed'].isna(), 'days_employed']=df_merge.loc[df['days_employed'].isna(), 'days_employed_right']

# Verificamos
df_merge.days_employed.isna().sum()

0

In [58]:
# Ahora ya no necesitamos esa columna, así que la eliminamos
df_merge=df_merge.drop(['days_employed_right'], axis=1)
df_merge

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,young adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,young adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,young adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,young adult
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21192,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,house,young adult
21193,0,9400.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,car,elderly
21194,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,house,young adult
21195,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,car,young adult


In [59]:
# Compruobamos que no tengamos valores ausentes
df_merge.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

In [60]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
print(df_merge.shape)
df_merge.days_employed

(21197, 13)


0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4        6600.000000
            ...     
21192    4529.316663
21193    9400.000000
21194    2113.346888
21195    3112.481705
21196    1984.507589
Name: days_employed, Length: 21197, dtype: float64

# 3. Clasificación de datos <a id='data_calsification'></a>

Aquí abordaremos los datos más importantes para analizar qué factores son los más recurrentes al momento de contraer una deuda. Según lo que ya conocemos, valores como la cantidad de hijos, el estado civil, y el ingreso familiar pueden ser valores importantes a analizar. Crearemos una categoría para 'total_income'. Una vez tengamos clasificados estos datos, analizaremos las hipótesis planteadas

In [61]:
# categorías de niveles de ingresos
def category_income(income):
    
    if income <10000:
        return'low'
    if income <35000:
        return 'average'
    if income <= 80000:
        return 'above average'
    if income <= 1250000:
        return 'high'
    return 'very high'


In [62]:
# Muestra los valores de los datos seleccionados para la clasificación

df_merge['category_income']=df_merge['total_income'].apply(category_income)
df_merge

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,category_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,house,young adult,above average
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car,young adult,average
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,house,young adult,average
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,young adult,above average
4,0,6600.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,adult,average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21192,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,house,young adult,above average
21193,0,9400.000000,67,secondary education,1,married,0,F,retiree,0,24959.969,car,elderly,average
21194,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,house,young adult,average
21195,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,car,young adult,above average


In [63]:
# Crea una columna con las categorías y cuenta los valores en ellas
df_merge.category_income.value_counts()

average          16202
above average     3847
low                926
high               222
Name: category_income, dtype: int64

# 4. Comprobación de las hipótesis <a id='hypotheses'></a>


**Hipótesis 1. ¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [64]:
# Comprueba los datos sobre los hijos y los pagos puntuales
hipo_1=df_merge[['children','debt']]
hipo_1.groupby('children').sum()

,debt
children,
0,1061
1,445
2,202
3,27
4,4
5,0


In [65]:
# Calculamos la tasa de incumplimiento en función del número de hijos
table_1 = hipo_1.groupby('children').agg({'debt':['count','sum']})
table_1['debt_rate'] = table_1[('debt','sum')]*100/table_1[('debt','count')]
table_1

debt        debt_rate
          count   sum           
children                        
0         13885  1061   7.641340
1          4816   445   9.240033
2          2118   202   9.537299
3           329    27   8.206687
4            40     4  10.000000
5             9     0   0.000000

**Conclusión**
En esta hipótesis podemos decir que se cumple  esto es porque si existen dedudas por parte de las personas con más hijos, sin embargo también tenemos un gran número de personas sin hijos con un deb_rate alto. Esto implica que las deudas que poseen estos clientes no están intimamente relacionadas a gastos de los hijos sino otro tipos de gastos como compra de bienes, deudas estudiantiles u otros.


**Hipótesis 2. ¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [67]:
# Comprobamos los datos del estado familiar y los pagos a tiempo
hipo_2=df_merge[['family_status','debt']]
hipo_2.groupby('family_status').sum()


,debt
family_status,
civil partnership,388
divorced,85
married,929
unmarried,274
widow / widower,63


In [68]:
# Calculamos la tasa de incumplimiento en función al estatus familiar
table_2 = hipo_2.groupby('family_status').agg({'debt':['count','sum']})
table_2['debt_rate'] = table_2[('debt','sum')]*100/table_2[('debt','count')]
table_2


debt      debt_rate
                   count  sum          
family_status                          
civil partnership   4126  388  9.403781
divorced            1194   85  7.118928
married            12140  929  7.652389
unmarried           2790  274  9.820789
widow / widower      947   63  6.652587

**Conclusión**

Podemos observar que las personas en unión civil poseen más porcentaje de deuda respecto a los demás, como segundo tenemos a las personas solteras. Esto confirma que nuestra suposición de que las personas casadas tienen mayores deudas es incorrecta.

**Hipótesis 3. ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [69]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
hipo_3=df_merge[['category_income','debt']]
hipo_3.groupby('category_income').sum()

,debt
category_income,
above average,276
average,1391
high,14
low,58


In [70]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
table_3 = hipo_3.groupby('category_income').agg({'debt':['count','sum']})
table_3['debt_rate'] = table_3[('debt','sum')]*100/table_3[('debt','count')]
table_3

debt       debt_rate
                 count   sum          
category_income                       
above average     3847   276  7.174422
average          16202  1391  8.585360
high               222    14  6.306306
low                926    58  6.263499

**Conclusión**
Aquí podríamos podemos observar que sí existe una relación, debido a que el porcentaje de endeudamiento de los que pertenecen a la categoría average es mayor que las demás

**Hipótesis 4. ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [71]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

hipo_4=df_merge[['purpose','debt']]
hipo_4.groupby('purpose').sum()

,debt
purpose,
car,402
education,370
house,488
real estate,293
wedding,186


In [72]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
table_4 = hipo_4.groupby('purpose').agg({'debt':['count','sum']})
table_4['debt_rate'] = table_4[('debt','sum')]*100/table_4[('debt','count')]
table_4

debt      debt_rate
            count  sum          
purpose                         
car          4272  402  9.410112
education    3964  370  9.334006
house        6842  488  7.132417
real estate  3813  293  7.684238
wedding      2306  186  8.065915

**Conclusión**

En este caso vemos que los propósitos de endeudamiento son muy variados y están bastante cercanos entre sí. Liderando la tabla tenemos que los clientes usan su crédito para obtener un auto o algo relacionado a la educación

# Conclusión general <a id='end'></a>

Nuestro proyecto consistía en preparar un informe para la división de préstamos de un banco. Donde se debía averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. 

En el Dataframe proporcionado encontramos valores ausentes y duplicados en primera instancia, por lo que se necesitó hacer ciertos cambios para un mejor análisis. Los cambios realizados se detallan a continuación.


1. Exploración de datos

Luego de explorar y conocer los datos que tenemos. Encontramos la misma cantidad de valores ausentes en dos columnas: days_employed y total_income. 
Los valores ausentes representan al: 10,09%
 Debido a que de todos los valores ingresados de 'secondary education', los que coinciden con valores ausentes en las columnas 'total_income' y 'days_employed' pertenecen al 7% del total de personas con valores faltantes. Además los 634 restantes también pertencen a un grupo muy pequeño. Podemos deducir que estos valores se encuentran ausentes por alguno(s) de los siguientes motivos:
 
-Los clientes no revelaron esta información por voluntad propia

-Los clientes no otorgaron esta información porque no estaban seguros de sus ingresos, ni sus días trabajados

-La información faltante no fue escrita en la base de datos por parte del Banco

2. Transformación de datos

Para continuar se necesitaba reprocesar los datos, los cambios que se hicieron fueron:

2.1. Revisar duplicados obvios y duplicados implícitos
Nos deshicimos de los duplicados obvios y nos ocupamos de los duplicados implícitos de la columna de education, donde se repetía secondary education en mayúscula. También creamos una función para colocar en purpose datos más estándar y no se repitieran los mismos motivos con diferentes palabras.

2.2. Nos ocupamos de los valores ausentes. 
Debido a que eran valores cuantitativos decidimos utilizar una clasificación de edad para utilizar la mediana que completarían los valores ausentes en total income y así obtener un valor más cercano a la realidad. Los valores que reemplazarán aquellos datos ausentes de days_employed, también fueron encontrados como datos representativos, aquí se creó una tabla que no solo nos sirvió para los datos ausentes sino también para los valores que sobrepasaban los 18250 días.

3. Otras modificaciones de datos: 

Repasamos las columnas, una por una, verificando que los datos en ellas sean correctos.

-En la columna days_employed vimos una gran cantidad de números negativos. Quizá hubo algún error al leer la cantidad, por lo que se procedió a cambiar a positivo. No podemos tener una cantidad negativa de días de trabajo.
-En la columna dob_years. Encontramos que uno de los valores dice 0, lo que sería imposible, porque representa la edad del cliente. Analizando en profundidad vemos que tenemos a 101 personas con 0 años lo que representa al (101x100)/21525=0,46% es una cantidad mínima, por lo que podríamos eliminar estos datos, sin embargo en el análisis de este informe NO influye la edad de las personas, pero SÍ sus ingresos, deudas, cantidad de hijos y otros factores. Rellenamos estos valores para evitar la pérdida de información relevante para el estudio.
-En la columna de gender, se cambió XNA por X debido a que X es la representación correcta para cuando una persona no se clasifica dentro de ningún género
-En la columna children, también modificamos dos valores:
-1 porque no es posible tener -1 hijos, lo cambiamos por 1 y 20, porque aunque es posible, tenemos a 76 personas con 20 hijos, parece un valor poco real. Por lo que se cambió 20 por 2.

Clasificamos los datos más importantes para analizar qué factores son los más recurrentes al momento de contraer una deuda. En este caso se hizo una categoría para el ingreso. 

**Prueba de hipótesis:**

1. ¿Existe una correlación entre tener hijos y pagar a tiempo?

La primera hipótesis es 50/50.

En esta hipótesis podemos decir que se cumple en cierta parte porque si existen deudas por parte de las personas con más hijos, sin embargo también tenemos un gran número de personas sin hijos con un deb_rate alto. Esto implica que las deudas que poseen estos clientes no están intimamente relacionadas a gastos de los hijos sino otro tipos de gastos como compra de bienes, deudas estudiantiles u otros.

2. ¿Existe una correlación entre la situación familiar y el pago a tiempo?

La segunda hipótesis ha sido aceptada.

Podemos observar que las personas en unión civil poseen más porcentaje de deuda respecto a los demás, como segundo tenemos a las personas solteras. Esto confirma que nuestra suposición de que las personas casadas tienen mayores deudas es incorrecta.

3. ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?

La tercera hipótesis ha sido aceptada completamente.

Aquí podríamos podemos observar que sí existe una relación, debido a que el porcentaje de endeudamiento de los que pertenecen a la categoría average es mayor que las demás

4. ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?

En este caso vemos que los propósitos de endeudamiento son muy variados y están bastante cercanos entre sí. Liderando la tabla tenemos que los clientes usan su crédito para obtener un auto o algo relacionado a la educación


Se concluye que el estado civil si afecta la capacidad de endeudamiento debido a la adquisición de bienes materiales o educación del cliente, sin embargo no se ha encontrado motivos suficientes para creer que la cantidad de hijos afecte al nivel de endeudamiento de los clientes, recapitulando vemos que mientras mayor números de hijos aumenta un poco el nivel de endeudamiento, sin embargo digo que no es suficiente ya que hemos denotado que muchas de las deudas no están intímamente relacionadas a la cantidad de hijos, por lo que vemos gran cantidad de personas sin hijos con índices de deuda altos también. Toda esta información se conlcuye de acuerdo a los datos correspondientes a la tabla completada sin valores ausentes.
